In [1]:
import pandas as pd
import keybert

In [10]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.data_getter import VKParser

In [11]:
owner_id='-129354225' # группа "центральный район за комфортную среду обитания"
token='...'

In [12]:
posts_df = VKParser().run_posts(owner_id, token, 100, "2024-03-01")

2024-02-01 19:30:00
posts downloaded


In [13]:
post_ids = posts_df["id"].to_list()
comments_df = VKParser().run_comments(owner_id=owner_id, post_ids=post_ids, access_token=token)

comments downloaded


In [15]:
result_df = VKParser().run_parser(owner_id, token, step=100, cutoff_date='2024-03-20') #объединяем посты и комментарии в один датафрейм

2024-02-01 19:30:00
posts downloaded
comments downloaded


In [16]:
result_df

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack
0,2024-03-21 19:30:00,333766,В доме на Синопской набережной 32/35 из-за поз...,795.0,28,8.0,post,NaN,NaN,NaN
1,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",3527.0,126,23.0,post,NaN,NaN,NaN
2,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,9866.0,188,24.0,post,NaN,NaN,NaN
3,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4599.0,54,8.0,post,"https://vk.com/wall-129354225_332885|ярко,",NaN,NaN
4,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,4355.0,51,11.0,post,https://vk.com/wall-129354225_325408|уплотните...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
715,2024-02-07 10:42:30,328420,"Люди десятилетиями ждут замены лифтов, чтоб он...",NaN,2,NaN,comment,NaN,328388.0,[]
716,2024-02-07 12:57:57,328422,"С вопросом капремонта лифтов, к сожалению, ст...",NaN,4,NaN,comment,NaN,328388.0,[]
717,2024-02-07 14:03:22,328427,Думаю что лифты в порядке. Просто их выключили...,NaN,1,NaN,comment,NaN,328388.0,[]
718,2024-02-07 15:26:01,328438,"[id10835085|Андрей], очень даже может быть",NaN,1,NaN,reply,NaN,328388.0,[328427]


In [17]:
result_df.to_csv("C:\\Users\\thebe\\Downloads\\ruina.csv")

In [153]:
df_from_csv = pd.read_csv("C:\\Users\\thebe\\Downloads\\ruina.csv")

In [154]:
from geocoder import Geocoder

g = Geocoder(device='cuda')

gdf = g.run(df_from_csv, 'text')

2024-03-22 14:11:45,652 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS
03-22 14:12 | INFO     | clear_names started
03-22 14:12 | INFO     | get_street started
03-22 14:12 | INFO     | extract_ner_street started


100%|██████████| 720/720 [00:04<00:00, 169.52it/s]

03-22 14:12 | INFO     | pattern1.sub started



100%|██████████| 720/720 [00:00<00:00, 719537.50it/s]

03-22 14:12 | INFO     | pattern2.findall started


...


100%|██████████| 720/720 [00:00<00:00, 36892.37it/s]

03-22 14:12 | INFO     | extract_toponym started



100%|██████████| 720/720 [00:00<00:00, 4140.13it/s]

03-22 14:12 | INFO     | get_stem started


ValueError: Unable to fill values because RangeIndex cannot contain NA

In [24]:
gdf

NameError: name 'gdf' is not defined

In [25]:
len(gdf['parents_stack'].iloc[187])

NameError: name 'gdf' is not defined

In [26]:
test_gdf = gdf.copy()

NameError: name 'gdf' is not defined

In [18]:
test_gdf

NameError: name 'test_gdf' is not defined

In [19]:
import numpy as np

for i in range(len(test_gdf)):
    tmp = test_gdf['parents_stack'].iloc[i]
    if type(tmp) is not float and len(tmp) > 0:
        test_gdf.at[i, 'parents_stack'] = tmp[0]
    else:
        test_gdf.at[i, 'parents_stack'] = np.nan


NameError: name 'test_gdf' is not defined

In [14]:
test_gdf = test_gdf.drop_duplicates(subset='id')
test_gdf = test_gdf.drop(columns=['date', 'text', 'views.count', 'likes.count', 'reposts.count', 'type', 'link', 'post_id', 'parents_stack'])
test_gdf = pd.merge(result_df, test_gdf, on='id', how='outer')
test_gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,...,Score_y,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,2024-02-01 19:30:00,327734,Как вам такая идея по обустройству широких газ...,3328.0,34,3.0,post,NaN,NaN,NaN,...,0.961,,Приморская,None,Приморская улица Санкт-Петербург Россия,['Приморская улица Санкт-Петербург Россия'],Приморская улица Санкт-Петербург Россия,Приморская улица,"Приморская улица, Просвещение, Петергоф, Санкт...",POINT (29.86224 59.89660)
1,2024-02-02 10:30:00,327851,Возвращение домофонов в парадные после капремо...,3746.0,58,4.0,post,NaN,NaN,NaN,...,0.987,60,Некрасова,None,улица Некрасова 60 Санкт-Петербург Россия,['улица Некрасова 60 Санкт-Петербург Россия'],улица Некрасова 60 Санкт-Петербург Россия,улица Некрасова,"60, улица Некрасова, Пески, округ Смольнинское...",POINT (30.36946 59.93851)
2,2024-02-02 07:48:48,327853,Повсеместно такая проблема.\nУгол Мытнинской и...,NaN,5,NaN,comment,NaN,327851.0,[],...,NaN,8,Мытнинской,None,"Мытнинская улица 8 Санкт-Петербург Россия,Мытн...","['Мытнинская улица 8 Санкт-Петербург Россия', ...",Мытнинская улица 8 Санкт-Петербург Россия,Мытнинская улица,"Мытнинская улица, 8-я Советская улица, Пески, ...",POINT (30.38073 59.93420)
3,2024-02-02 08:24:25,327854,"Можно закидать меня тряпками, но я сторонник у...",NaN,13,NaN,comment,NaN,327851.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,2024-02-02 08:31:37,327855,"Некрасова 60, 4-ая парадная. Установлена новая...",NaN,7,NaN,comment,NaN,327851.0,[],...,0.992,60,Некрасова,None,улица Некрасова 60 Санкт-Петербург Россия,['улица Некрасова 60 Санкт-Петербург Россия'],улица Некрасова 60 Санкт-Петербург Россия,улица Некрасова,"60, улица Некрасова, Пески, округ Смольнинское...",POINT (30.36946 59.93851)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2024-03-21 16:33:06,333767,"Вредит не застройка, а косорукие строители и п...",NaN,1,NaN,comment,NaN,333766.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
478,2024-03-21 16:33:49,333768,"Ужас, сквозные щели! Страшно жить в таком доме!",NaN,1,NaN,comment,NaN,333766.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
479,2024-03-21 16:36:27,333770,статью заменят на хулиганку?,NaN,0,NaN,comment,NaN,333530.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
480,2024-03-21 16:38:47,333771,Что и требовалось ожидать...,NaN,0,NaN,comment,NaN,333766.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [20]:
len(result_df)

720

In [22]:
import random
import numpy as np
index_list = []


result_df['toponim'] = np.nan
for i in range (50):
    index_list.append(random.randint(0, len(result_df)))

index_list

[211,
 141,
 169,
 252,
 158,
 442,
 1,
 73,
 447,
 690,
 490,
 277,
 29,
 163,
 559,
 520,
 621,
 612,
 114,
 422,
 183,
 358,
 270,
 526,
 135,
 601,
 631,
 639,
 587,
 345,
 266,
 476,
 620,
 211,
 682,
 141,
 416,
 170,
 355,
 242,
 442,
 717,
 206,
 85,
 137,
 579,
 451,
 487,
 223,
 636]

In [23]:
test_gdf = result_df.copy()

for i in index_list:
    test_gdf.at[i, 'toponim'] = f'toponim_{i}'

In [135]:
test_gdf.at[121, 'toponim'] = 'toponim_implemented_1'

In [136]:
test_gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,toponim
0,2024-03-21 19:30:00,333766,В доме на Синопской набережной 32/35 из-за поз...,795.0,28,8.0,post,NaN,NaN,NaN,NaN
1,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",3527.0,126,23.0,post,NaN,NaN,NaN,toponim_1
2,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,9866.0,188,24.0,post,NaN,NaN,NaN,NaN
3,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4599.0,54,8.0,post,"https://vk.com/wall-129354225_332885|ярко,",NaN,NaN,NaN
4,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,4355.0,51,11.0,post,https://vk.com/wall-129354225_325408|уплотните...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
715,2024-02-07 10:42:30,328420,"Люди десятилетиями ждут замены лифтов, чтоб он...",NaN,2,NaN,comment,NaN,328388.0,NaN,NaN
716,2024-02-07 12:57:57,328422,"С вопросом капремонта лифтов, к сожалению, ст...",NaN,4,NaN,comment,NaN,328388.0,NaN,NaN
717,2024-02-07 14:03:22,328427,Думаю что лифты в порядке. Просто их выключили...,NaN,1,NaN,comment,NaN,328388.0,NaN,toponim_717
718,2024-02-07 15:26:01,328438,"[id10835085|Андрей], очень даже может быть",NaN,1,NaN,reply,NaN,328388.0,328427,NaN


In [119]:
import numpy as np

for i in range(len(test_gdf)):
    tmp = test_gdf['parents_stack'].iloc[i]
    if type(tmp) is not float and len(tmp) > 0:
        test_gdf.at[i, 'parents_stack'] = tmp[0]
    else:
        test_gdf.at[i, 'parents_stack'] = np.nan

TypeError: object of type 'int' has no len()

In [120]:
check = test_gdf['toponim'].loc[test_gdf['id'] == 333766][0]
type(check) is float

True

Список id постов с топонимами

In [137]:
post_top_gdf = test_gdf.loc[test_gdf['type'] == 'post']
post_top_gdf = post_top_gdf.dropna(subset='toponim')
post_toponim_list = list(post_top_gdf['id'])
post_toponim_list

[333530, 331686, 329242, 328572]

Список id комментариев с топонимами

In [138]:
comment_top_gdf = test_gdf.loc[test_gdf['type'] == 'comment']
comment_top_gdf = comment_top_gdf.dropna(subset='toponim')
comment_toponim_list = list(comment_top_gdf['id'])
comment_toponim_list

[333775,
 333532,
 333535,
 333565,
 333596,
 333698,
 332892,
 332895,
 332927,
 332598,
 332283,
 330212,
 330217,
 329443,
 329448,
 329451,
 329528,
 329550,
 329561,
 329752,
 328941,
 328977,
 328984,
 329005,
 329014,
 329025,
 328833,
 328427]

Список id реплаев с топонимами

In [139]:
reply_top_gdf = test_gdf.loc[test_gdf['type'] == 'reply']
reply_top_gdf = reply_top_gdf.dropna(subset='toponim')
reply_toponim_list = list(reply_top_gdf['id'])
reply_toponim_list

[333540,
 333606,
 333669,
 333717,
 333765,
 333782,
 333604,
 332973,
 332957,
 332744,
 331181,
 331194,
 330246,
 328975,
 328991,
 328404]

In [140]:
exclude_list = reply_toponim_list + comment_toponim_list

In [147]:
for i in post_toponim_list:
    df_to_extract = list((test_gdf['text'].loc[(test_gdf['post_id'] == i)
                                         & (~test_gdf['id'].isin(exclude_list))
                                         &(~test_gdf['parents_stack'].isin(comment_toponim_list))]))

    df_to_extract.append(test_gdf['text'].loc[test_gdf['id'] == i])

    print(i, len(df_to_extract))

333530 98
331686 1
329242 1
328572 1


In [151]:
for i in comment_toponim_list:
    df_to_extract = list(test_gdf['text'].loc[test_gdf['parents_stack'] == i])

    df_to_extract.append(test_gdf['text'].loc[test_gdf['id'] == i])

    print(i, len(df_to_extract))

333775 1
333532 6
333535 1
333565 1
333596 1
333698 1
332892 7
332895 1
332927 8
332598 1
332283 1
330212 1
330217 1
329443 1
329448 1
329451 1
329528 1
329550 1
329561 2
329752 1
328941 1
328977 1
328984 1
329005 1
329014 1
329025 1
328833 1
328427 2


In [152]:
for i in reply_toponim_list:
    df_to_extract = test_gdf['text'].loc[test_gdf['id'] == i]

    print(i, len(df_to_extract))

333540 1
333606 1
333669 1
333717 1
333765 1
333782 1
333604 1
332973 1
332957 1
332744 1
331181 1
331194 1
330246 1
328975 1
328991 1
328404 1


In [127]:
test_gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,toponim
0,2024-03-21 19:30:00,333766,В доме на Синопской набережной 32/35 из-за поз...,795.0,28,8.0,post,NaN,NaN,NaN,NaN
1,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",3527.0,126,23.0,post,NaN,NaN,NaN,toponim_1
2,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,9866.0,188,24.0,post,NaN,NaN,NaN,NaN
3,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4599.0,54,8.0,post,"https://vk.com/wall-129354225_332885|ярко,",NaN,NaN,NaN
4,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,4355.0,51,11.0,post,https://vk.com/wall-129354225_325408|уплотните...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
715,2024-02-07 10:42:30,328420,"Люди десятилетиями ждут замены лифтов, чтоб он...",NaN,2,NaN,comment,NaN,328388.0,NaN,NaN
716,2024-02-07 12:57:57,328422,"С вопросом капремонта лифтов, к сожалению, ст...",NaN,4,NaN,comment,NaN,328388.0,NaN,NaN
717,2024-02-07 14:03:22,328427,Думаю что лифты в порядке. Просто их выключили...,NaN,1,NaN,comment,NaN,328388.0,NaN,toponim_717
718,2024-02-07 15:26:01,328438,"[id10835085|Андрей], очень даже может быть",NaN,1,NaN,reply,NaN,328388.0,328427,NaN


In [15]:
# Предполагается, что df - это ваш DataFrame с данными
# Структура df: ['id', 'text', 'type', 'parent_id', 'parents_stack', 'toponim']

# Функция для добавления id комментариев в новую ячейку
def link_comments(df):
    # Создаем новый столбец для связанных id комментариев
    df['linked_comments'] = None

    # Перебираем строки с типом 'post'
    for index, post in df[df['type'] == 'post'].iterrows():
        linked_ids = []

        # Проверяем наличие топонима в посте
        if pd.isnull(post['only_full_street_name']):
            # Собираем комментарии, относящиеся к этому посту
            comments = df[(df['type'] == 'comment') & (df['post_id'] == post['id'])]
            for _, comment in comments.iterrows():
                # Если в комментарии есть топоним, добавляем его id и id всех комментариев, ссылающихся на него
                if pd.notnull(comment['only_full_street_name']):
                    linked_ids.append(comment['id'])
                    linked_ids.extend(df[df['parents_stack'] == comment['id']]['id'].tolist())
        else:
            # Собираем все комментарии, относящиеся к посту с топонимом
            comments = df[(df['type'] == 'comment') & (df['post_id'] == post['id'])]
            for _, comment in comments.iterrows():
                # Добавляем id комментариев, если у них нет топонима и они не ссылаются на комментарий с топонимом
                if pd.isnull(comment['only_full_street_name']) and pd.isnull(df[df['id'] == comment['parents_stack']]['only_full_street_name']).all():
                    linked_ids.append(comment['id'])

        # Записываем связанные id комментариев в новую ячейку
        if linked_ids:
            df.at[index, 'linked_comments'] = ', '.join(map(str, linked_ids))

    return df

# Применяем функцию к DataFrame
df = test_gdf  # Замените это на ваш DataFrame
df = link_comments(df)

# Выводим обновленный DataFrame
df

ValueError: ('Lengths must match to compare', (482,), (0,))

In [ ]:
df

In [ ]:
df = df.drop_duplicates(subset='id')
df